# AM207 Final Paper - Decomposition of Uncertainty in Bayesian Deep Learning

### Team Members - **add research goal for Spring if wanted**

* Owen Callen
* Gabriel Pestre
* Hayden Sansum
* Nikhil Vanderklaauw

## Problem Statement

The authors of this paper \[1\] propose a method of utilizing the flexibility of a Bayesian Neural Network with Latent Variables to allow for the decomposition of Aleatoric and Epistemic Uncertainty. When performing supervised learning or reinforcement learning there are a number of ways to gain insight into the overall uncertainty, such as frequentist bootstrapping (running multiple trials) or implementing a Bayesian modeling approach to incorporate a distribution over models. Both these approaches however only provide the overall magnitude of the uncertainty and do not allow for further interrogation of where those uncertainties might be coming from. Without knowing how much of the variation is due to inherent randomness (aleatoric) or due to the lack of collected data in specific areas (epistemic) there is no way to know precisely how to improve the model. For instance would it be useful to collect more data in a certain area? Would it be useful to defer the prediction to an expert? In reinforcement learning this could be framed as, where does my agent need to explore more to maximize how much it learns about the environment? Being able to decompose uncertainty into these two core components, epistemic and aleatoric (whilst still using a complex and flexible model such as a neural network) enables practioners of data science and statistics to tackle these problems. 

Aside from this primary aim, of decomposing uncertainty, the authors are also interested in proving that the abililty to achieve this decomposition of uncertainty in this way is an inherent property of the Bayesian Neural Network with Latent Variables (BBN+LV) (originally presented in an early paper by the author \[2\]) and not an artefact of the sampling methodology. As such they attempt to show that the BNN+LV has the properties required to split epistemic from aleatoric uncertainty when using both BB-alpha (a similar method to BBVI although using an alpha divergence) and also Hamiltonian Monte Carlo (HMC).

## Context/Scope

* Why is this important?
* Why does splitting aleatoric from epistemic uncertainty matter?
* How have they framed it in terms of heteroscedastic vs bimodal noise
* What about latent variables - baby weights etc.
* Reinforcement learning - active learning - where to explore
* Novel risk sensitive criterion?

## Existing Work

* **Our paper**: Depeweg - Decomposition of Uncertainty in Bayesian Deep Learning for Efficient Risk-sensitive learning (https://arxiv.org/pdf/1710.07283.pdf)
  * Self explanatory - this is our main paper

* BNN+LV paper: Depeweg - Learning and Policy Search in Stochastic Dynamical Systems With Baysian Neural Networks (https://arxiv.org/pdf/1605.07127.pdf)
  * This is the original Depeweg paper where they introduce the idea of using BNN+LV in the first place

* Overcoming non-identifiability in BNN+LVs: Wei Wei Pan - Learning Deep Bayesian Latent Variable Regression Models that
Generalize: When Non-identifiability is a Problem (https://arxiv.org/pdf/1911.00569.pdf)
  * A later paper by Prof Pan and a few others which highlights some of the problems with the original BNN+LV uncertainty paper in that its almost impossible to train

* Sensitivity Analysis: Depeweg - Sensitivity Analysis for Predictive Uncertainty
in Bayesian Neural Networks (https://arxiv.org/pdf/1712.03605.pdf)
  * Depeweg also published a paper looking into the sensitivity of the results of the orignal paper so we should pull out of few key findings from here

* Bayes by Backprop (BBN - BBVI): Blundell - Weight Uncertainty in Neural Networks (https://arxiv.org/pdf/1505.05424.pdf)
  * The OG BBVI for NN implementation - we should mention and credit this as the paper which enabled use to perform approximate inference on Neural Network posteriors

* Applications of the decomposition in image detection: Kendall - What Uncertainties Do We Need in Bayesian Deep
Learning for Computer Vision? (https://arxiv.org/pdf/1703.04977.pdf)
  * A specific application of the BBN_LV models to CV - i think they expand on the toy examples with some real heteroscedastic noise examples

* Uncertainty through Entropy Decomposition: Kozachenko - Sample Estimate of the Entropy of a Random Vector (http://www.mathnet.ru/php/archive.phtml?wshow=paper&jrnid=ppi&paperid=797&option_lang=eng)
  * The old paper demonstrating how you can use entropy to get uncertainty in a dataset (its in Russian but we only used the maths)

* A newer easy to digest version of the above with additional details we ignorewd: Singh - Nearest Neighbor Estimates of Entropy (https://www.tandfonline.com/doi/pdf/10.1080/01966324.2003.10737616?needAccess=true)


Interestingly, in our search for existing literature on 

* Robertson et al (2014). Quantifying Epistemic and Aleatoric Uncertainty in the Ampair 600 Wind Turbine. SAND2014-20179C. Available at: [https://www.osti.gov/servlets/purl/1242753]

## Contribution

The contribution of this paper is threefold and centers on the uniqueness of utilizing a BNN+LV with uncertainty decompositions:

1. The specific methods (described below in our technical section) of decomposing the predictive uncertainty of a BNN+LV into epistemic and aleatoric uncertainties by using entropy and variance. The algorithm used to calculate the entropy of a set of data has been around since the original Kozachenko paper in 1987 \[8\] but this application of it to a BBN+LV  

## Technical Content

### BBN+LV

### Entropy Decomposition 

To distinguish between the epistemic and aleatoric uncertainties, the authors required a method to decompose where the uncertainties are being introduced - either the uncertainties across the weights (epistemic) or the uncertainty inherent within the noise of the data (aleatoric). 

In the original Depeweg paper \[1\] the authors proposed two ways to perform this estimation, via entropy and via variance. In the results section of the paper, the authors only present and discuss their results in terms of the entropy and so this is the method that we will focus on reproducing. The entropy of a set of points represent how much inherent variation exists. Formally this can be calculated using the functional pdf, but in this case where our BNN+LV posterior is not a known form, it must be approximated using a technique such as K-Nearest Neighbor.

Using KNN as an approximate Entropy calculation goes back to a 1987 paper by Kozachenko & Leonenko \[7\]
although is more succinctly stated in a later 2003 paper by Singh et al \[8\] with the relevant equation being listed as (Eqn. 4). Entropy as a concept is actually a measurement of randomness and is leverged in many fields from thermodynamics to algorithms which optimize splits in a decision tree. In this specific case, the randomness (entropy) that is interesting is how much difference there is in the predictions at each input data point. K-Nearest Neighbors is therefore an obvious candidate for an approximation as it enables an efficient and fast method to calculate distances between a vector of datapoint (a set of $y*$ predictions for a given $x*$). 

This KNN approximation of the Entropy, in general form, is stated as:
$$ 
\hat{H}_n = \frac{p}{n}\sum_{i=1}^n ln(p_i) + ln\left[ \frac{\pi^{p/2}}{\Gamma(\frac{p}{2} + 1)} \right] + \gamma + ln(n-1)
$$

Where:
$$ 
p = \text{dimensionality} \quad n = \text{number of data points} \quad p_i = \text{distance to nearest neighbor} \quad \gamma = \text{Euler constant}
$$

So in essence the entropy is the average of the natural log of the distance from each point to its closest neighbour, scaled by the size and dimensionality of the problem at hand. The additional constants are related to the surface area of a hypersphere in $p$ dimensions and scales the entropy based on the overall size of the hypersphere within which the entropy is being calculated. 

Singh et al [\8\] also propose more robust methods of approximation utilizing more than just the single nearest neighbouring point but for simplicity of reproduction and to more closely match the Depeweg paper \[1\], which cites the original Kozachenko paper \[8\], we will stick to the single nearest neighbor method.

With a method for approximating the Entropy of a given set of data, we must then apply this to the BNN+LV in order to decompose the uncertainty. Naively applying the entropy calculation across the outputs of the BNN+LV however will not give us anything more than the overall uncertainty, including both aleatoric and epistemic. To decompose this we need to calculate two entropies, the overall entropy and the entropy which occurs having marginalized out the uncertainty over the weights (aleatoric uncertainty). The difference of which leaves just the epistemic uncertainty.

The formulation given in the paper for this decomposition is:
$$
\text{Overall uncertainty} = H(y*|x*)
$$
$$
\text{Aleatoric uncertainty} = \mathbb{E}_{q(W)}[H(y*|W,x*)]
$$
$$
\text{Epistemic uncertainty} = H(y*|x*) - \mathbb{E}_{q(W)}[H(y*|W,x*)]
$$

Being unable to calculate the entropies exactly though, we instead substitute in our approximation for the entropy, $\hat{H}(.)$ and calculate it as follows:
$$
\text{Overall uncertainty} = \hat{H}(y*^1,...,y*^L)
$$
$$
\text{Aleatoric uncertainty} = \frac{1}{M} \sum_{i=1}^M \hat{H}(y*^{1,W_i}, ..., y*^{L,W_i})
$$
$$
\text{Epistemic uncertainty} = \hat{H}(y*^1,...,y*^L) - \frac{1}{M} \sum_{i=1}^M \hat{H}(y*^{1,W_i}, ..., y*^{L,W_i})
$$

Where:
$$ 
L = \text{number of data points} \quad M = \text{number of different models (posterior samples)} 
$$

In simpler terms:

1. Use the KNN formula to calculate the overall entropy of a given set of output data points, $y*$ from all the models at once, this treats all predictions from each model, at a specific data point $x*$ as a single vector. This calculates the overall entropy present in the predictions.
2. Make multiple predictions using a single set of model weights, $W_i$ and calculate the entropy at each $y*$ within this model. Repeat this process for every different model individually ($W_i,...,W_M$) and average the entropy across these outcomes. By separately calculating the average of the entropy across many different models through repeated sampling of the weights, this averages out the effects of the uncertainty between models (aka marginalizes out the epistemic uncertainties leaving just the aleatoric).
3. Finally summarize the Epistemic uncertainty as $Total - Aleatoric$

### BBVI

### BB-alpha

### HMC & PYMC3

## Experiments

We need to HIGHLIGHT:
* At least one instance of it working - aka we can kinda replicate the results and explain how we did it in detail and why
* At least one instance of it failing - where did we find it difficult to replicate or run into problems?

### Heteroscedastic Noise

### Bimodal Noise

### Wet Chicken Dynamics

## Comparing Results

## Evaluation

## Future Work & Improvements

## Broader Impact 

## References

\[1\] Depeweg et al (2018). Decomposition of Uncertainty in Bayesian Deep Learning for Efficient Risk-sensitive learning. arXiv:1710.07283v4. Available at: [https://arxiv.org/pdf/1710.07283.pdf]


\[2\] Depeweg et al (2017). Learning and Policy Search in Stochastic Dynamical Systems With Baysian Neural Networks. arXiv:1605.07127v3. Available at: [https://arxiv.org/pdf/1605.07127.pdf]

\[3\] Yacoby, Pan and Doshi-Velez (2019). Learning Deep Bayesian Latent Variable Regression Models that Generalize: When Non-identifiability is a Problem. arXiv:1911.00569. Available at: [https://arxiv.org/pdf/1911.00569.pdf]
  
\[4\] Depeweg et al (2017). Sensitivity Analysis for Predictive Uncertainty in Bayesian Neural Networks. arXiv:1712.03605v1. Available at: [https://arxiv.org/pdf/1712.03605.pdf]

\[5\] Blundell et al (2015). Weight Uncertainty in Neural Networks. arXiv:1505.05424v2. Available at: [https://arxiv.org/pdf/1505.05424.pdf]
  
\[6\] Kendall and Gal (2017). What Uncertainties Do We Need in Bayesian Deep Learning for Computer Vision? arXiv:1703.04977v2. Available at: [https://arxiv.org/pdf/1703.04977.pdf]
  
\[7\] Kozachenko and Leonenko (1987). Sample Estimate of the Entropy of a Random Vector. Probl. Peredachi Inf., 1987, Volume 23, Issue 2, Pages 9–16. Available at: [http://www.mathnet.ru/php/archive.phtml?wshow=paper&jrnid=ppi&paperid=797&option_lang=eng]

\[8\] Singh et al (2003). Nearest Neighbor Estimates of Entropy. American Journal of Mathematical and
Management Sciences. 23:3-4, 301-321. Available at: [https://doi.org/10.1080/01966324.2003.10737616]

\[9\] Robertson et al (2014). Quantifying Epistemic and Aleatoric Uncertainty in the Ampair 600 Wind Turbine. SAND2014-20179C. Available at: [https://www.osti.gov/servlets/purl/1242753]